In [21]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
from PyPDF2 import PdfFileWriter, PdfFileReader 

In [33]:
def abc(a):
    if type(a) == str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else:
        z=a
    return z

def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False    

def icici_p1(f):
    tables=tabula.read_pdf(f,
                              lattice=True,
                              pages="all",
                              silent=True,
                              multiple_tables=True,
                              pandas_options={'header':None})
    df = pd.DataFrame()
    df = pd.concat([c for c in tables]).drop_duplicates()
    
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)

    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('transaction').any(),axis=1)==True].index][0]
            df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
        except:
            try:
                idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('remarks').any(), axis=1) ==True].index ][0]
                df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
            except:
                print("\nICICI Column Headers Missing"); pass
    
    try:
        df = df.loc[:, df.columns.notnull()]
    except:pass
    
    print(df.columns)
    
    try:
        mode=[c for c in df.columns if "MODE" in str(c).upper() ][0]
        print("\nMode columns present")
        verif = "mode"   
    except: 
        verif = "nomode"
        print("\nMode columns missing")
    
    #print(df)
    return df,verif


    
def icici_p1_process(df):
    
    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
        df[bal]=df[bal].apply( lambda x: abc(x) )
    except: print("\nBalance columns missing")
        
    try:
        amt=[c for c in df.columns if "AMOUNT" in str(c).upper() ][0]
        df[[bal,amt]]=df[[bal,amt]].replace(r'^\s*$', np.nan, regex=True)
        try:
            df["Debits"] = np.nan; df["Credits"] = np.nan;
            for j,i in enumerate(df[amt]):
                if "Cr" in str(i):
                    df["Credits"][j] = df[amt][j]
                elif "Dr" in str(i):
                    df["Debits"][j] = df[amt][j]            
        except Exception as e: 
            print("Error",e)
    except:pass        

    try:
        dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
        except:
            try:
                df["CHQ"] = np.nan
                chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
            except:pass
    #print(chq)
            

    try:
        narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
                except:
                    try:
                        narr=[c for c in df.columns if "NARRATION" in str(c).upper() ][0]
                    except:pass
    #print(narr)

    try:
        wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
        except: pass 
    #print(wdl)

    try:
        dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
    except:
        try:
            dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
        except: pass
    #print(dep)
    #print(df.head())

    try:
        df[[wdl, dep]] = df[[wdl, dep]].replace({"NA":np.nan, "-":np.nan,"0":np.nan, "0.00":np.nan})
    except:pass
    
    #df.drop_duplicates(keep = False, inplace = True)
    
    #print(df.head(20))
    
    
    df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
    df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
    df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)


    df = df[[dat,chq,narr,wdl,dep,bal]]
    df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]
    
    df["Xns Date"] = df["Xns Date"].str.replace("\r","")
        
    df.to_csv("check.csv", index=False)    
        
    df["Xns Date"]=pd.to_datetime(df["Xns Date"], dayfirst=True)
    df["Xns Date"]=df["Xns Date"].dt.date.astype(str)
    
    if df["Xns Date"][0]>df["Xns Date"][len(df)-1]:
        df = df.iloc[::-1]
        df.reset_index(inplace = True, drop =True)
    
    df.to_excel("check.xlsx", index=False)
    return df

def icici_p3(f,num):
    tmp = pd.DataFrame()
    tables = camelot.read_pdf(f, pages = str(num), flavor = "stream", edge_tol = 25)
    tmp = tables[0].df
    return tmp

def icici_p3_process(df):
    try:
        idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
        df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
        print("1st attempt")
    except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
        except:
            print("\nAxis-Column headers missing"); pass

    drop_idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index]
    #print(drop_idx)
    #print(len(drop_idx))
    if len(drop_idx)>0:
        df.drop(df.index[drop_idx],inplace=True)
    
    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
    except: print("\nBalance columns missing")

    try:
        dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
        except:
            try:
                dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
            except:pass

    try:
        chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
    except:
        try:
            chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
        except:
            try:
                df["CHQ"] = np.nan
                chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
            except:pass

    try:
        narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
                except:
                    try:
                        narr=[c for c in df.columns if "NARRATION" in str(c).upper() ][0]
                    except:pass

    try:
        for j,i in enumerate(df[narr]):
            if isnan(i) == True:
                df[narr][j] = df[narr][j-1]+df[narr][j+1]
            else:
                pass
    except:pass

    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)

    try:
        wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
    except:
        try:
            wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
        except: pass 

    try:
        dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
    except:
        try:
            dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
        except: pass

    df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
    df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
    df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)


    df = df[[dat,chq,narr,wdl,dep,bal]]
    df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]

    df["Xns Date"] = df["Xns Date"].str.replace("\r","")
    
    list1 = []
    for j,i in enumerate(df["Xns Date"]):
            if isnan(i) == True:
                list1.append(j)
    if len(list1)>0:
        df.drop(df.index[list1],inplace=True)

    df["Xns Date"]=pd.to_datetime(df["Xns Date"], dayfirst=True)
    df["Xns Date"]=df["Xns Date"].dt.date.astype(str)
    
    if df["Xns Date"][0]>df["Xns Date"][len(df)-1]:
        df = df.iloc[::-1]
        df.reset_index(inplace = True, drop =True)
    
    df.to_excel("check.xlsx", index=False)
    return df



f = r"C:\Users\rites\Desktop\mudracircle\bks_raw\Parsing_testing\ICICI\ICICI\pattern_2\icici03.pdf"
two_df = icici_p1(f)
df = two_df[0]

if two_df[1] == "mode":
    pdf = PdfFileReader(open(f,'rb'))
    total_cnt = pdf.getNumPages()

    dt = pd.DataFrame();
    for i in range(1,total_cnt + 1):
        tmp = icici_p3(f, i)
        if tmp.shape[1] == 6:
            tmp = pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
            dt = pd.concat([dt, tmp]).reset_index(drop=True)
    df = icici_p3_process(dt)
    
elif two_df[1] == "nomode":
    df = icici_p1_process(df)
    
print("Parsed")

Index(['Sr No', 'Value Date', 'Transactio\rn Date', 'Cheque\rNumber',
       'Transactio\rn Remarks', 'Debit\rAmount', 'Credit\rAmount',
       'Balance(IN\rR)'],
      dtype='object', name=0)

Mode columns missing


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:160]
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:164]
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:165]


Parsed
